In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Exploracion de datos

In [2]:
# CAMBIAR, HACER RELATIVOS NO SE COMO
# Mientras poner tu propia ruta
PATH = '/home/max/workspace/NASA_Spaceapps_2024/hello_flask/data/DEF_SRDB_V5_1827_5-20241005_212220/srdb-data-V5.csv'
RELATIVE_PATH = 'hello_flask/data/DEF_SRDB_V5_1827_5-20241005_212220/srdb-data-V5.csv'
DATADIR_PATH = '/home/max/workspace/NASA_Spaceapps_2024/hello_flask/data'

to_keep = [
    'Entry_date', 
    'Quality_flag', 
    'Country', 
    'Region', 
    'Site_name', 
    'Site_ID', 
    'Study_midyear', 
    'Species', 
    'Biome', 
    'Ecosystem_type',
    'Leaf_habit', 
    'Soil_type', 
    'Soil_drainage', 
    'MAT', 
    'MAP', 
    'PET', 
    'Study_temp', 
    'Study_precip'
]

to_drop = [
    'Entry_date',
    'Quality_flag',
    'Ecosystem_type', 
    'Soil_type', 
    'PET', 
    'Study_temp', 
    'Study_precip', 
    'Site_ID'
]

# Leer el archivo CSV y seleccionar solo las columnas especificadas
df = pd.read_csv(PATH, usecols=to_keep, parse_dates=['Entry_date']).drop(columns=to_drop).dropna()

# Reorganizar las columnas para que 'Study_midyear' sea la primera
columns = ['Study_midyear'] + [col for col in df.columns if col != 'Study_midyear']
df = df[columns]

# Ordenar el DataFrame por 'Study_midyear'
df = df.sort_values(by='Study_midyear').reset_index(drop=True)

# Exportar los datos
df.to_csv(DATADIR_PATH+'/Processed/1_clean_srdb-data-V5.csv', index=False)

## dropNA ?????

Cuantos paises tenemos?

In [34]:
#df = df.dropna()

In [35]:
df['Country'].unique()

array(['USA', 'Congo', 'United Kingdom', 'Brazil', 'Canada', 'Venezuela',
       'Japan', 'India', 'China', 'Turkey', 'Norway', 'Denmark',
       'Thailand', 'Germany', 'UK', 'Belgium', 'Italy', 'Russia',
       'Malawi', 'Finland', 'Ireland', 'Malaysia', 'Costa Rica',
       'Australia', 'South Korea', 'Italy ', 'Czech Republic', 'Mexico',
       'Spain', 'Philippines', 'Sweden', 'Panama', 'Indonesia', 'Poland',
       'Isreal', 'Zambia', 'Argentina', 'New Zealand', 'Tunisia',
       'Mongolia', 'Kenya', 'Peru', 'Switzerland', 'Greenland', 'Hungary',
       'Taiwan', 'Cesko', 'France', 'Puerto Rico', 'Portugal', 'Vietnam',
       'Singapore', 'Guinea', 'Netherlands', 'Bennin', 'Signapore',
       'Zimbabwe', 'Czechia', 'Chile', 'Estonia', 'South Africa',
       'Austria', 'Tanzania'], dtype=object)

### Cuantas especies?

In [36]:
species = []
for spec in df['Species']: 
    #print(spec.split(';'))
    species.extend(spec.split(';'))
    
species_unique = pd.Series(pd.Series(species).unique())

species_unique


0                                          Pinus echinata
1                                             Acer rubrum
2                                                     Mix
3                                            Quercus alba
4                                      Thuja occidentalis
                              ...                        
1098                              Juniperus phoenaicea L.
1099    Salix oritrepha, Spiraea alpina Pallas, Potent...
1100                    Camelia sinensis var. sinensis L.
1101                                        Sphagnum spp.
1102                               Achnatherum hymenoides
Length: 1103, dtype: object

## Especies de interes para agricultores

* Oryza sativa - Arroz
* Zea mays - Maíz
* Triticum aestivum - Trigo
* Glycine max - Soya
* Solanum tuberosum - Papa

In [37]:
from difflib import get_close_matches

cultivos = ['Oryza sativa', 'Zea mays', 'Triticum aestivum', 'Glycine max', 'Solanum tuberosum']
matches = {}

for species in species_unique:
    match = get_close_matches(species, cultivos, n=1, cutoff=0.6)  # Coincidencias con un umbral de similitud
    if match:
        matches[species] = match[0]  # Guarda la especie del cultivo si hay coincidencia

matches  # Ver las coincidencias encontradas


{'Triticum aestivum': 'Triticum aestivum',
 'Pennisetum clandestinum': 'Triticum aestivum',
 'Glycine max': 'Glycine max',
 'Triticum aestivum, Zea mays subsp.': 'Triticum aestivum',
 'Zea mays': 'Zea mays',
 ' Avena sativa': 'Oryza sativa',
 'Oryza sativa': 'Oryza sativa',
 'Zea mays subsp.': 'Zea mays',
 ' Zea mays': 'Zea mays',
 ' Protium sagotianum': 'Triticum aestivum',
 'Protium sagotianum': 'Triticum aestivum',
 'Zea mays, triticum aestivum': 'Triticum aestivum',
 'Zea mays, Glycine max': 'Glycine max',
 ' Oryza sativa': 'Oryza sativa',
 ' Glycine max': 'Glycine max',
 ' Triticum aestivum': 'Triticum aestivum',
 'Pisum sativum': 'Triticum aestivum',
 ' Pisum sativum': 'Triticum aestivum',
 'Zea mays L.': 'Zea mays',
 'Triticum, zea mays': 'Triticum aestivum',
 'Sphagnum fuscum': 'Solanum tuberosum',
 'Zea Mays L.': 'Zea mays',
 'Oryza sativa, Brassica napus': 'Oryza sativa',
 'Triticum aestivum, Zea mays': 'Triticum aestivum',
 'Oryza sativa, zea mays': 'Oryza sativa',
 'Triticu

traducimos en el dataframe

In [38]:
# Diccionario de traducción
traducciones = {
    'Oryza sativa': 'Arroz',
    'Zea mays': 'Maíz',
    'Triticum aestivum': 'Trigo',
    'Glycine max': 'Soya',
    'Solanum tuberosum': 'Papa'
}

# Función para aplicar la traducción
def traducir_especies(cultivo):
    for clave, valor in traducciones.items():
        if clave in cultivo:
            return valor
    return cultivo  # Si no encuentra coincidencia, devuelve el nombre original

# Aplicar la traducción a los valores del DataFrame
#df['Species_traducida'] = df['Species'].apply(traducir_especies)

# Ver el resultado
#df[['Species', 'Species_traducida']]

# Aplicar la traducción a los valores del DataFrame
df['Species_traducida'] = df['Species'].apply(traducir_especies)

# Filtrar las filas donde 'Species_traducida' es diferente de 'Species' (es decir, fue traducido)
df_traducido = df[df['Species'] != df['Species_traducida']]

# Ver el resultado final
df_traducido[['Species', 'Species_traducida']]

df_traducido['Species_traducida'].unique()

array(['Trigo', 'Soya', 'Maíz', 'Arroz'], dtype=object)

cuantos datos tenemos de cada especie de interes?

In [39]:
df_traducido['Species_traducida'].value_counts()

Species_traducida
Maíz     209
Trigo    164
Arroz     90
Soya      80
Name: count, dtype: int64

## Agrupamiento (sin filtro de especie)

In [40]:
df.head()

,Study_midyear,Country,Region,Site_name,Species,Biome,Leaf_habit,Soil_drainage,MAT,MAP,Species_traducida
0,1961.0,USA,Tennessee,Oak Ridge,Pinus echinata,Temperate,Evergreen,Dry,14.0,1500.0,Pinus echinata
1,1961.0,USA,Tennessee,Oak Ridge,Acer rubrum,Temperate,Deciduous,Dry,14.0,1500.0,Acer rubrum
2,1961.0,Congo,Yangambi,Esali,Mix,Tropical,Evergreen,Dry,24.4,1875.0,Mix
3,1961.0,USA,Tennessee,Oak Ridge,Quercus alba,Temperate,Deciduous,Dry,14.0,1500.0,Quercus alba
4,1962.0,USA,Tennessee,Oak Ridge,Quercus alba,Temperate,Deciduous,Dry,14.0,1500.0,Quercus alba


In [41]:
df.to_csv(DATADIR_PATH+'/Processed/2_species-trad_srdb-data-V5.csv', index=False)

In [42]:
df[df['Country'] == 'Mexico']

,Study_midyear,Country,Region,Site_name,Species,Biome,Leaf_habit,Soil_drainage,MAT,MAP,Species_traducida
1780,2006.0,Mexico,Yucatan,El Eden Ecological Preserve,Mix,Tropical,Evergreen,Dry,24.2,1650.0,Mix
1938,2007.0,Mexico,Yucatan,El Eden Ecological Preserve,Mix,Tropical,Evergreen,Dry,24.2,1650.0,Mix
2821,2011.0,Mexico,Veracruz,Estero Dulce,Pachira aquatica Aubl,Tropical,Deciduous,Wet,27.0,1425.0,Pachira aquatica Aubl
2822,2011.0,Mexico,Veracruz,Estero Dulce,"Thalia geniculata L.,",Tropical,Deciduous,Wet,27.0,1425.0,"Thalia geniculata L.,"
2823,2011.0,Mexico,Veracruz,Laguna Chica,Pachira aquatica Aubl,Tropical,Deciduous,Wet,27.0,1425.0,Pachira aquatica Aubl
2824,2011.0,Mexico,Veracruz,Laguna Chica,Cyperus giganteus,Tropical,Deciduous,Wet,27.0,1425.0,Cyperus giganteus
2825,2011.0,Mexico,Veracruz,Boquilla de Oro,Ficus insipida,Tropical,Deciduous,Wet,27.0,1425.0,Ficus insipida
2826,2011.0,Mexico,Veracruz,Boquilla de Oro,Pontederia sagittata,Tropical,Deciduous,Wet,27.0,1425.0,Pontederia sagittata


In [43]:
# Definir las funciones de agregación personalizadas
agg_funcs = {
    'Country': 'first',
    'Region': 'first',
    'Study_midyear': 'first',
    'Site_name': 'first',
    'Species': lambda x: ', '.join(x.unique()),
    'Biome': 'first',
    'Leaf_habit': 'first',
    'Soil_drainage': 'first',
    'MAT': 'mean',
    'MAP': 'mean'
}

# Agrupar por 'Site_name' y 'Study_midyear' y aplicar las funciones de agregación
grouped_df = df.groupby(['Site_name', 'Study_midyear']).agg(agg_funcs).reset_index(drop=True)

# Exportar los datos
grouped_df.to_csv(DATADIR_PATH+'/Processed/2_grouped-no-trad_srdb-data-V5.csv', index=False)


### Comparamos 

df agrupado: 

In [44]:
grouped_df[grouped_df['Country'] == 'Mexico']

,Country,Region,Study_midyear,Site_name,Species,Biome,Leaf_habit,Soil_drainage,MAT,MAP
205,Mexico,Veracruz,2011.0,Boquilla de Oro,"Ficus insipida, Pontederia sagittata",Tropical,Deciduous,Wet,27.0,1425.0
510,Mexico,Yucatan,2006.0,El Eden Ecological Preserve,Mix,Tropical,Evergreen,Dry,24.2,1650.0
511,Mexico,Yucatan,2007.0,El Eden Ecological Preserve,Mix,Tropical,Evergreen,Dry,24.2,1650.0
519,Mexico,Veracruz,2011.0,Estero Dulce,"Pachira aquatica Aubl, Thalia geniculata L.,",Tropical,Deciduous,Wet,27.0,1425.0
962,Mexico,Veracruz,2011.0,Laguna Chica,"Pachira aquatica Aubl, Cyperus giganteus",Tropical,Deciduous,Wet,27.0,1425.0


df sin agrupar: 

In [45]:
df[df['Country'] == 'Mexico']

,Study_midyear,Country,Region,Site_name,Species,Biome,Leaf_habit,Soil_drainage,MAT,MAP,Species_traducida
1780,2006.0,Mexico,Yucatan,El Eden Ecological Preserve,Mix,Tropical,Evergreen,Dry,24.2,1650.0,Mix
1938,2007.0,Mexico,Yucatan,El Eden Ecological Preserve,Mix,Tropical,Evergreen,Dry,24.2,1650.0,Mix
2821,2011.0,Mexico,Veracruz,Estero Dulce,Pachira aquatica Aubl,Tropical,Deciduous,Wet,27.0,1425.0,Pachira aquatica Aubl
2822,2011.0,Mexico,Veracruz,Estero Dulce,"Thalia geniculata L.,",Tropical,Deciduous,Wet,27.0,1425.0,"Thalia geniculata L.,"
2823,2011.0,Mexico,Veracruz,Laguna Chica,Pachira aquatica Aubl,Tropical,Deciduous,Wet,27.0,1425.0,Pachira aquatica Aubl
2824,2011.0,Mexico,Veracruz,Laguna Chica,Cyperus giganteus,Tropical,Deciduous,Wet,27.0,1425.0,Cyperus giganteus
2825,2011.0,Mexico,Veracruz,Boquilla de Oro,Ficus insipida,Tropical,Deciduous,Wet,27.0,1425.0,Ficus insipida
2826,2011.0,Mexico,Veracruz,Boquilla de Oro,Pontederia sagittata,Tropical,Deciduous,Wet,27.0,1425.0,Pontederia sagittata


### Nos quedamos con las que tengan cierto numero de "CONDICION"

In [46]:
CONDICION = 'Region'

In [47]:
df[CONDICION].value_counts() > 5

Region
Massachusetts      True
Inner Mongolia     True
Ontario            True
Shaanxi            True
North Carolina     True
                  ...  
Gelderland        False
Carlow            False
Gifu              False
Cologne           False
Borneo            False
Name: count, Length: 297, dtype: bool

In [48]:
# Contar las ocurrencias de cada país
country_counts = df[CONDICION].value_counts()

# Filtrar los países que tienen más de 100 ocurrencias
countries_to_keep = country_counts[country_counts > 5].index

# Filtrar el DataFrame original para mantener solo esos países
filtered_df = df[df[CONDICION].isin(countries_to_keep)]

filtered_df[CONDICION].unique()

array(['Tennessee', 'England', 'Minnesota', 'Amazonas', 'Missouri',
       'South Carolina', 'Saskatchewan', 'Alaska', 'Utah', 'Oklahoma',
       'North Carolina', 'Oregon', 'Washington', 'New Mexico',
       'Massachusetts', 'Ontario', 'Florida', 'Puerto Rico', 'Haryana',
       'Michigan', 'California', 'Quebec', 'Kansas', 'Georgia', 'Hawaii',
       'Wisconsin', 'Scotland', 'Rondondia', 'Texas', 'Maine', 'New York',
       'Beijing', 'Ohio', 'Manitoba', 'Arizona', 'Alberta', 'Artvin',
       'Indiana', 'Greenland', 'Montana', 'Iowa', 'Québec',
       'Kanchanaburi', 'Shannxi', 'North Dakota', 'Inner Mongolia',
       'Tibet', 'Colorado', 'British Columbia', 'Jiangxi', 'Solling',
       'Qinghai', 'Virginia', 'Wyoming', 'NE Wales', 'Nova Scotia',
       'Guangdong', 'Lampang', 'Pará', 'Fujian', 'Henan', 'Jilin',
       'Zomba', 'Agro-Environmental Sciences station', 'Sichuan',
       'South Dakota', 'Nebraska', 'Yunnan', 'Mato Grosso', 'Hunan',
       'Kannus', 'Shandong', 'Jiangsu',

In [49]:
filtered_df[CONDICION].value_counts()

Region
Massachusetts     157
Inner Mongolia    129
Ontario           122
Shaanxi           110
North Carolina    102
                 ... 
Şanlıurfa          6
Sicily              6
Nasu                6
Nanjing             6
Kericho             6
Name: count, Length: 146, dtype: int64

In [50]:
# Obtener el conteo de ocurrencias de cada país
country_counts = grouped_df['Country'].value_counts()

# Mostrar el top de países con el número de ocurrencias
top_countries = country_counts.head(10)  # Puedes ajustar el número de países en el top

print(top_countries)

Country
USA               680
China             514
Canada            156
Germany            50
Brazil             43
Japan              34
Spain              19
UK                 17
Czech Republic     16
Finland            16
Name: count, dtype: int64


In [51]:
grouped_df[grouped_df['Country'] == 'USA']['Study_midyear'].count()

680